importamos las librerías

In [22]:
pip install numpy



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
import pandas as pd
import numpy as np
import json
import ast
from textblob import TextBlob
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt 
import sys
import locale

Leemos el archivo y creamos un DataFrame

In [14]:
# Abrimos el archivo output_steam_games para pasarlo a df

df = pd.read_parquet('D:\Potato\LABS Henry\PI_ML_OPS\ETL-EDA\Archivos\output_steam_games.parquet')

In [15]:
df_games = df

In [16]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,nan,None,None,None
1,None,None,None,None,None,None,None,None,None,nan,None,None,None
2,None,None,None,None,None,None,None,None,None,nan,None,None,None
3,None,None,None,None,None,None,None,None,None,nan,None,None,None
4,None,None,None,None,None,None,None,None,None,nan,None,None,None


In [6]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   publisher     24083 non-null   object
 1   genres        28852 non-null   object
 2   app_name      32133 non-null   object
 3   title         30085 non-null   object
 4   url           32135 non-null   object
 5   release_date  30068 non-null   object
 6   tags          31972 non-null   object
 7   reviews_url   32133 non-null   object
 8   specs         31465 non-null   object
 9   price         120445 non-null  object
 10  early_access  32135 non-null   object
 11  id            32133 non-null   object
 12  developer     28836 non-null   object
dtypes: object(13)
memory usage: 11.9+ MB


Buscamos nulos y duplicados

In [19]:
df_games.isna().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price               0
early_access    88310
id              88312
developer       91609
dtype: int64

In [24]:
#convertimos los valores nan a NaN para poder eliminar los nulos
df_games['price']= df_games['price'].replace('nan', np.nan)

In [25]:
#al tener tantos datos nulos se decidió eliminarlos 
df_games= df_games.dropna(how='all').reset_index(drop=True)


In [27]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,None


In [31]:
#verificamos duplicados
df_games[df_games['id'].duplicated(keep=False)]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


In [30]:
#eliminamos duplicados
df_games.drop_duplicates(subset='id', keep='first', inplace=True)
df_games.dropna(subset='id', inplace=True)

In [32]:
df_games.shape

(32132, 13)

Transformamos los datos

In [33]:
df_games.dtypes

publisher       object
genres          object
app_name        object
title           object
url             object
release_date    object
tags            object
reviews_url     object
specs           object
price           object
early_access    object
id              object
developer       object
dtype: object

El dataset aun tiene el tipo de dato object, asi que aun puede contener datos anidados. Analizamos por medio de un bucle las columnas y almacenamos en un diccionario vacio para transformalo en un dataset

In [34]:
datos= {'columna':[], 'tipo_dato':[]} #diccionario vacio

for columna in df_games.columns: #bucle para recorrer las columnas
    datos['columna'].append(columna)
    datos['tipo_dato'].append(df_games[columna].apply(type).unique())

a = pd.DataFrame(datos)
a


,columna,tipo_dato
0,publisher,"[<class 'str'>, <class 'NoneType'>]"
1,genres,"[<class 'numpy.ndarray'>, <class 'NoneType'>]"
2,app_name,"[<class 'str'>, <class 'NoneType'>]"
3,title,"[<class 'str'>, <class 'NoneType'>]"
4,url,[<class 'str'>]
5,release_date,"[<class 'str'>, <class 'NoneType'>]"
6,tags,"[<class 'numpy.ndarray'>, <class 'NoneType'>]"
7,reviews_url,[<class 'str'>]
8,specs,"[<class 'numpy.ndarray'>, <class 'NoneType'>]"
9,price,"[<class 'str'>, <class 'float'>]"


Transformamos columna 'release_date'

In [36]:
# extraemos el año de la columna 'release_date y reemplazamos nulos por sin dato
df_games['Año'] = df_games['release_date'].str.extract(r'(\d{4})').fillna('sin dato')
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Año
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
4,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,None,sin dato
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",2018
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,2018
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,2018
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",2017


Transformamos la columna 'price'


In [37]:
df_games['price']

0                4.99
1        Free To Play
2        Free to Play
3                0.99
4                2.99
             ...     
32130            1.99
32131            4.99
32132            1.99
32133            4.99
32134            4.99
Name: price, Length: 32132, dtype: object

In [38]:
#ya que la columna price es de tipo objeto debemos transformarla a tipo float
#definimos una función para reemplazar los datos 

def cambio_float(dato):
    if pd.isna(dato):
        return 0.0
    try:
        flotante = float(dato) #si el valor es de tipo float se conserva
        return flotante
    except (ValueError, TypeError): #si el valor es diferente retornará 0.0
        return 0.0
    
#aplicamos la función a la columna 'price'
df_games['price'] = df_games['price'].apply(cambio_float)

In [41]:
#comprobamos
df_games['price']

0        4.99
1        0.00
2        0.00
3        0.99
4        2.99
         ... 
32130    1.99
32131    4.99
32132    1.99
32133    4.99
32134    4.99
Name: price, Length: 32132, dtype: float64

Transformamos la columna 'genres'

In [47]:
df_games['genres'][0]

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
      dtype=object)

In [48]:
df_games = df_games.explode('genres')
df_games = df_games.dropna(subset=['genres'])
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Año
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
0,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
0,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
0,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
0,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018


Completamos datos nulos o vacios

In [49]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71551 entries, 0 to 32133
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     59112 non-null  object 
 1   genres        71551 non-null  object 
 2   app_name      71549 non-null  object 
 3   title         71549 non-null  object 
 4   url           71551 non-null  object 
 5   release_date  71498 non-null  object 
 6   tags          71505 non-null  object 
 7   reviews_url   71551 non-null  object 
 8   specs         70994 non-null  object 
 9   price         71551 non-null  float64
 10  early_access  71551 non-null  object 
 11  id            71551 non-null  object 
 12  developer     71201 non-null  object 
 13  Año           71551 non-null  object 
dtypes: float64(1), object(13)
memory usage: 8.2+ MB


In [50]:
#variable para almancenar las columnas y reemplazar los nulos
columnas = ['publisher', 'app_name', 'title', 'developer']

#realizamos dataframe con las columnas y nulos para reemplazar por 'sin dato'Archivos

sd= df_games[columnas].fillna('Sin Dato')

#eliminamos las columnas y concatenamos los DF
df_games = pd.concat([df_games.drop(columnas, axis=1), sd], axis= 1)
df_games.head()

,genres,url,release_date,tags,reviews_url,specs,price,early_access,id,Año,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


Guardamos el dataset como csv

In [56]:
df_games.to_csv('Archivos/steam_games.csv', index= False, encoding= 'utf-8')

In [62]:
#convertimos a parquet para la optimización de los archivos
df_games = pd.read_csv('D:\Potato\LABS Henry\PI_ML_OPS\ETL-EDA\Archivos\steam_games.csv')

parquet_file = 'Archivos/steam_parquet.parquet' 

#transformamos mediante una tabla el archivo csv en parquet
tabla = pa.Table.from_pandas(df_games)
pq.write_table(tabla, parquet_file)